In [1]:
!pip install shap

     |████████████████████████████████| 371 kB 17.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for shap: filename=shap-0.40.0-cp37-cp37m-linux_x86_64.whl size=509231 sha256=c1d96d6750d13468ed4246dfc08d3179b23d071294199a90124d254411edd1fa
  Stored in directory: /root/.cache/pip/wheels/ec/35/84/e304841ac4b910bc95fe9a6e5302eb2507b4333728851dcbfb
Successfully built shap


In [2]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
import xgboost as xg
import matplotlib.pyplot as plt
from matplotlib import pyplot
from xgboost import plot_importance
import seaborn as sb
import shap
from sklearn.model_selection import cross_validate
from sklearn import metrics 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [106]:
df=pd.read_csv("/content/ime692_project.csv")

In [ ]:
#prof = ProfileReport(df)

/usr/local/lib/python3.7/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


FOR EDA OF dataset

In [ ]:
#prof

Number of variables,19
Number of observations,756
Total Missing (%),0.0%
Total size in memory,112.3 KiB
Average record size in memory,152.2 B
Numeric,15
Categorical,2
Boolean,2
Date,0
Text (Unique),0
Rejected,0


TO Remove high cardinality from column county


In [4]:
from collections import Counter
def cumulatively_categorise(column,threshold=0.2,return_categories_list=True):
  #Find the threshold value using the percentage and number of instances in the column
  threshold_value=int(threshold*len(column))
  #Initialise an empty list for our new minimised categories
  categories_list=[]
  #Initialise a variable to calculate the sum of frequencies
  s=0
  #Create a counter dictionary of the form unique_value: frequency
  counts=Counter(column)

  #Loop through the category name and its corresponding frequency after sorting the categories by descending order of frequency
  for i,j in counts.most_common():
    #Add the frequency to the global sum
    s+=dict(counts)[i]
    #Append the category name to the list
    categories_list.append(i)
    #Check if the global sum has reached the threshold value, if so break the loop
    if s>=threshold_value:
      break
  #Append the category Other to the list
  categories_list.append('Other')

  #Replace all instances not in our new categories by Other  
  new_column=column.apply(lambda x: x if x in categories_list else 'Other')

  #Return transformed column and unique values if return_categories=True
  if(return_categories_list):
    return new_column,categories_list
  #Return only the transformed column if return_categories=False
  else:
    return new_column


#Call the function with a default threshold of 75%
transformed_column,new_category_list=cumulatively_categorise(df['County'],return_categories_list=True)



In [86]:
len(new_category_list)

36

In [127]:
df_train=df[df['Test']==0]
df_test=df[df['Test']==1]

In [128]:
df_train['country']=transformed_column
df_test['country']=transformed_column


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [129]:
df_train.drop('County',axis=1,inplace=True)
df_test.drop('County',axis=1,inplace=True)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [130]:
df_train

,State,IT_WholeRate,HighSchool_WholeRate,MedianInc_WholeAvg,republican_rate,Segregation,urban,racial_weighted_bias,hesitancy,HighSchool_Disparity,IT_Disparity,MedianInc_Disparity,CvdVax_DisparityY,vehicle,FacNumRate,CaseRate,Black_Prop,Test,country
0,California,91.4,88.4,99.406,1770.559475,49.890444,1,33.258827,9,5.7,10.7,62.678,16.768796,90.4,0.017292,5.114553,11.031221,0,Other
1,California,89.1,89.2,52.537,4775.484604,62.099507,1,36.520664,13,0.7,6.4,24.189,13.494706,92.8,0.017793,5.288203,1.897019,0,Other
2,California,86.5,71.3,59.401,5726.667504,19.219010,0,37.223305,14,18.7,-0.6,39.137,15.707145,94.6,0.018564,10.205597,1.308767,0,Other
4,California,84.0,80.1,45.283,5641.809291,35.769303,0,36.879187,15,21.5,-12.6,-57.687,23.012579,90.4,0.028765,4.631094,3.523659,0,Other
5,California,90.1,93.4,83.377,5324.527717,58.879477,1,36.525529,10,8.4,3.6,-4.821,7.449124,95.9,0.017112,5.069927,1.037113,0,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,Wisconsin,83.3,93.7,53.986,4439.702475,73.099649,1,36.775647,17,15.1,-6.2,-0.735,20.113383,92.7,0.041715,10.801854,1.253766,0,Other
752,Wisconsin,87.6,90.8,60.652,6476.031146,65.224838,0,35.934490,17,12.3,0.2,26.775,23.430382,94.9,0.013661,14.317103,3.345894,0,Other
753,Wisconsin,91.4,95.8,73.893,2293.396669,49.608129,1,35.577848,14,8.4,9.0,42.958,26.019744,92.7,0.016645,8.236402,5.531969,0,Other
754,Wisconsin,89.8,92.1,62.340,5282.318703,58.983900,1,37.440183,17,12.5,6.1,40.201,26.188287,94.5,0.018145,13.017971,2.976465,0,Other


In [131]:
x_train=df_train.drop('CvdVax_DisparityY',axis=1)
y_train=df_train['CvdVax_DisparityY']
x_test=df_test.drop('CvdVax_DisparityY',axis=1)
y_test=df_test['CvdVax_DisparityY']

In [132]:
y_train

0      16.768796
1      13.494706
2      15.707145
4      23.012579
5       7.449124
         ...    
751    20.113383
752    23.430382
753    26.019744
754    26.188287
755    16.316005
Name: CvdVax_DisparityY, Length: 531, dtype: float64

In [111]:
x_train_std=x_train.drop(['country','State'],axis=1)
x_test_std=x_test.drop(['country','State'],axis=1)

In [112]:
x_train_temp=x_train[['country','State']]
x_test_temp=x_test[['country','State']]

In [113]:
for column in x_train_std:
    x_train_std[column] = (x_train_std[column] - x_train_std[column].mean()) / x_train_std[column].std()    

In [114]:
for column in x_test_std:
    x_test_std[column] = (x_test_std[column] - x_test_std[column].mean()) / x_test_std[column].std()    

In [117]:
xf_train= pd.concat([x_train_std,temp],axis=1,join='outer')
xf_test= pd.concat([x_test_std,temp],axis=1,join='outer')

In [118]:
xf_train= pd.get_dummies(xf_train, columns = ['State', 'country'])
xf_test= pd.get_dummies(xf_test, columns = ['State', 'country'])

baseline model without hyperparemeter tuning

In [119]:
model_1 = xg.XGBRegressor(objective ='reg:linear',n_estimators = 10, seed = 123)

In [121]:
y_train

0      16.768796
1      13.494706
2      15.707145
4      23.012579
5       7.449124
         ...    
751    20.113383
752    23.430382
753    26.019744
754    26.188287
755    16.316005
Name: CvdVax_DisparityY, Length: 531, dtype: float64

In [120]:
model_1.fit(xf_train,y_train)

[18:05:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBoostError: ignored

In [17]:
pred = model_1.predict(test_X)

In [18]:
rmse = np.sqrt(MSE(test_y, pred))
print("RMSE : % f" %(rmse))

RMSE :  8.442700


In [19]:
print(model_1.feature_importances_)

[0.02818874 0.00726981 0.0077268  0.12051285 0.17831825 0.
 0.02182476 0.03735738 0.         0.02331958 0.         0.00731237
 0.07940883 0.06724054 0.02585639 0.         0.         0.13533007
 0.10684253 0.         0.066374   0.04265376 0.         0.
 0.         0.         0.         0.         0.         0.03452696
 0.         0.         0.         0.         0.         0.00993644
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]


In [20]:
key=train_X.columns
val=model_1.feature_importances_

In [21]:
feature_importance= pd.DataFrame()

In [22]:
feature_importance['features']=key
feature_importance['values']=val

Top 20 features based on gain 

In [23]:
feature_importance.nlargest(20, columns='values')

,features,values
4,Segregation,0.178318
17,State_Illinois,0.135330
3,republican_rate,0.120513
18,State_Indiana,0.106843
12,FacNumRate,0.079409
13,CaseRate,0.067241
20,State_New Jersey,0.066374
21,State_New York,0.042654
7,hesitancy,0.037357
29,State_Virginia,0.034527


In [29]:
imp=list(feature_importance.features)

In [24]:
feature_importance.plot(x="features", y=["values"], kind="bar")

In [25]:
shap.initjs()

# explain the model's predictions using SHAP values

Shap value to gain furthur insight about the dataset for both locally and globally

In [47]:
imp[0:10]

['IT_WholeRate',
 'HighSchool_WholeRate',
 'MedianInc_WholeAvg',
 'republican_rate',
 'Segregation',
 'urban',
 'racial_weighted_bias',
 'hesitancy',
 'HighSchool_Disparity',
 'IT_Disparity']

In [69]:
xf_shap=xf[imp[0:14]]

In [70]:
xf_shap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756 entries, 0 to 755
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   IT_WholeRate          756 non-null    float64
 1   HighSchool_WholeRate  756 non-null    float64
 2   MedianInc_WholeAvg    756 non-null    float64
 3   republican_rate       756 non-null    float64
 4   Segregation           756 non-null    float64
 5   urban                 756 non-null    float64
 6   racial_weighted_bias  756 non-null    float64
 7   hesitancy             756 non-null    float64
 8   HighSchool_Disparity  756 non-null    float64
 9   IT_Disparity          756 non-null    float64
 10  MedianInc_Disparity   756 non-null    float64
 11  vehicle               756 non-null    float64
 12  FacNumRate            756 non-null    float64
 13  CaseRate              756 non-null    float64
dtypes: float64(14)
memory usage: 82.8 KB


In [73]:
explainer = shap.TreeExplainer(model_1)
shap_values = explainer.shap_values(X=xf_shap)

shap.summary_plot(shap_values, xf_shap, show=False)
plt.tight_layout();

ValueError: ignored

In [62]:
shap.summary_plot(shap_values, xf, plot_type="bar",show=False)
plt.tight_layout();

Transform dataset for hyperparemeter tuning


In [ ]:
dmatrix = xg.DMatrix(data=xf, label=y)

Series.base is deprecated and will be removed in a future version
Series.base is deprecated and will be removed in a future version


In [ ]:
params={ 'objective':'reg:squarederror',
         'max_depth': 6, 
         'colsample_bylevel':0.5,
         'learning_rate':0.01,
         'random_state':20}

In [ ]:
cv_results = xg.cv(dtrain=dmatrix, params=params, nfold=10, metrics={'rmse'}, as_pandas=True, seed=20,num_boost_round=1000)

In [ ]:
print('RMSE: %.2f' % cv_results['test-rmse-mean'].min())

RMSE: 6.15


Best model tested on different hyperparemeters values based on random search cv

In [ ]:
params = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': [0.01, 0.1, 0.2, 0.3],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 500, 1000]}

In [ ]:
xgbr = xg.XGBRegressor(seed = 20)
model_3 = RandomizedSearchCV(estimator=xgbr,
                         param_distributions=params,
                         scoring='neg_mean_squared_error',
                         n_iter=25,
                         verbose=1)
model_3.fit(train_X, train_y)

In [ ]:
print("Best parameters:", model_3.best_params_)
print("Lowest RMSE: ", (-model_3.best_score_)**(1/2.0))

Best parameters: {'subsample': 0.5, 'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.1, 'colsample_bytree': 0.6, 'colsample_bylevel': 0.6}
Lowest RMSE:  6.788543999069327


In [ ]:
y_pred=model_3.predict(test_X)

In [ ]:
val_rmse = np.sqrt(MSE(test_y, y_pred))

In [ ]:
val_rmse

6.308947221208206